<a href="https://colab.research.google.com/github/piyukhadse/RAG_Pipeline/blob/main/complete_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving extracted_text_data.json to extracted_text_data.json


In [4]:
import io
file_name = "extracted_text_data.json"

# Open the file, read its content, and decode it as UTF-8
with open(file_name, 'r', encoding='utf-8') as f:
    file_content = f.read()

# Create a StringIO object from the file content
# (optional, if you need to work with it as an in-memory stream)
string_io_object = io.StringIO(file_content)

# You can now work with the string_io_object or directly with file_content

In [5]:
print(file_name)

extracted_text_data.json


In [6]:
import json

In [7]:
extracted_data = json.load(open(file_name))

In [8]:
from transformers import AutoTokenizer, AutoModel
import torch

In [9]:
# Load multilingual model (e.g., XLM-RoBERTa)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModel.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [10]:
def embed_text(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=80)
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings

In [11]:
# Embed extracted text
embeddings = {}
for lang, docs in extracted_data.items():
    embeddings[lang] = []
    for doc in docs:
        doc_embedding = embed_text(doc["text"])
        embeddings[lang].append({"filename": doc["filename"], "embedding": doc_embedding})

In [12]:
from transformers import pipeline

# Load a multilingual summarization model
summarizer = pipeline("summarization", model="facebook/mbart-large-50", device=0)

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Device set to use cuda:0


In [13]:
# Summarize extracted text
def summarize_text(text):
    chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
    summaries = [summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text'] for chunk in chunks]
    return " ".join(summaries)

In [14]:
def clean_text(text):
    """
    Cleans the input text by removing unwanted characters, Unicode sequences,
    and repetitive patterns.
    """
    # Remove newline characters and backslashes
    text = text.replace("\n", " ").replace("\\", " ")

    # Remove specific unwanted sequences
    text = re.sub(r"\|\|", " ", text)  # Remove ||
    text = re.sub(r"\[U\+200E\]", " ", text)  # Remove [U+200E]

    # Remove specific dashes and repetitive patterns
    text = re.sub(r"[-—–]{2,}", " ", text)  # Sequences of dashes
    text = re.sub(r"[=]{2,}", " ", text)  # Sequences of equal signs

    # Remove unwanted Unicode characters like \u200f, \u200e, and similar
    text = re.sub(r"[\u200f\u200e]", "", text)

    # Remove sequences of Chinese/Japanese-like characters (一 一 一, etc.)
    text = re.sub(r"[\u4e00-\u9fff]+", " ", text)

    # Remove multiple spaces
    text = re.sub(r"\s{2,}", " ", text)

    # Strip leading and trailing whitespace
    text = text.strip()

    return text

In [15]:
def preprocess_data(data):
    """
    Cleans the text data in a nested dictionary format.
    Args:
        data (dict): Dictionary containing text data to preprocess.

    Returns:
        dict: Preprocessed data with cleaned text.
    """
    cleaned_data = {}
    for lang, documents in data.items():
        cleaned_documents = []
        for doc in documents:
            cleaned_text = clean_text(doc['text'])
            cleaned_documents.append({"filename": doc['filename'], "text": cleaned_text})
        cleaned_data[lang] = cleaned_documents
    return cleaned_data

In [16]:
import re

In [17]:
preprocessed_data = preprocess_data(extracted_data)

In [ ]:
print(preprocessed_data)

In [24]:
pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.37
    Uninstalling SQLAlchemy-2.0.37:
      Successfully uninstalled SQLAlchemy-2.0.37
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [33]:
from datasets import Dataset

def chunk_text(text, chunk_size=1000):
    """Split text into chunks of a specified size."""
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

def prepare_dataset(text_list):
    """Prepare a Hugging Face Dataset from a list of texts."""
    # Flatten text into chunks for processing
    chunks = [{"text": chunk} for text in text_list for chunk in chunk_text(text)]
    return Dataset.from_list(chunks)

In [34]:
def summarize_dataset(dataset, batch_size=16):
    """Summarize text using batched processing."""
    def generate_summary(batch):
        summaries = summarizer(batch["text"], max_length=130, min_length=30, truncation=True)
        # Perform summarization on the batch
        return {"summary": [{"summary_text": s["summary_text"]} for s in summaries]}
    # Apply summarization to the dataset in batches
    summarized_dataset = dataset.map(generate_summary, batched=True, batch_size=batch_size)
    return summarized_dataset

In [35]:
def combine_summaries(text_list, summarized_dataset):
    """Group summaries back by original document."""
    chunk_counts = [len(chunk_text(text)) for text in text_list]
    grouped_summaries = []
    idx = 0
    for count in chunk_counts:
        summaries = " ".join(s["summary_text"] for s in summarized_dataset[idx:idx + count]["summary"])
        grouped_summaries.append(summaries)
        idx += count
    return grouped_summaries

In [36]:
summaries = {}
for lang, docs in preprocessed_data.items():
    text_list = [doc["text"] for doc in docs]

    # Prepare dataset
    dataset = prepare_dataset(text_list)

    # Summarize in batches
    summarized_dataset_result = summarize_dataset(dataset)

    # Combine summaries
    final_summaries = combine_summaries(text_list, summarized_dataset_result)

    # Store summaries
    summaries[lang] = [{"filename": doc["filename"], "summary": summary} for doc, summary in zip(docs, final_summaries)]


Map:   0%|          | 0/1111 [00:00<?, ? examples/s]

Your max_length is set to 130, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 130, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)


Map:   0%|          | 0/764 [00:00<?, ? examples/s]

Your max_length is set to 130, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 130, but your input_length is only 64. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Your max_length is set to 130, but your input_length is only 14. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 130, but your input_length is only 65. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)


Map:   0%|          | 0/369 [00:00<?, ? examples/s]

In [ ]:
 print(summarized_dataset)

Dataset({
    features: ['text', 'summary'],
    num_rows: 1111
})


In [37]:
# Save summaries
with open("summaries.json", "w", encoding="utf-8") as f:
    json.dump(summaries, f, ensure_ascii=False, indent=4)

In [38]:
# Question Answering
qa_pipeline = pipeline("question-answering", model="xlm-roberta-large")

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Device set to use cuda:0


In [39]:

def answer_question(question, context):
    return qa_pipeline(question=question, context=context)

In [40]:
# Example Q&A
question = " A challenge organizations face is how to renew blue oceans?"
answers = {}
for lang, docs in summaries.items():
    answers[lang] = []
    for doc in docs:
        answer = answer_question(question, doc["summary"])
        answers[lang].append({"filename": doc["filename"], "answer": answer})

# Save answers
with open("answers.json", "w", encoding="utf-8") as f:
    json.dump(answers, f, ensure_ascii=False, indent=4)

In [42]:
print(answers)

{'en': [{'filename': 'Blue_Ocean_Strategy,_Expanded_Edition_How_to_Create_Uncontested-2.pdf', 'answer': {'score': 8.801071089692414e-05, 'start': 166564, 'end': 166603, 'answer': 'of Fame by Fast Company magazine and as'}}, {'filename': 'Reboot_Leadership_and_the_Art_of.pdf', 'answer': {'score': 2.4130673409672454e-05, 'start': 103238, 'end': 103292, 'answer': 'By payment of the required fees, you have been granted'}}, {'filename': 'The Alchemist by Paulo Coelho-1.pdf', 'answer': {'score': 9.066352504305542e-05, 'start': 47008, 'end': 47069, 'answer': 'He had used the two stones only once, one morning when he was'}}], 'ur': [{'filename': '12-Rabiul-Awal-2024.pdf', 'answer': {'score': 5.655158747686073e-05, 'start': 677, 'end': 683, 'answer': 'جال ۓگ'}}, {'filename': 'Extension-of-Ahdoc-Employees.pdf', 'answer': {'score': 9.646119724493474e-05, 'start': 237, 'end': 248, 'answer': 'اعدہ(1)(دا)'}}, {'filename': 'fasana-e-ajaib final.pdf', 'answer': {'score': 3.190920324414037e-05, 'start'